# Content

[1. Setup](#Setup)

[2. Data Cleaning and Preparation](#Data-Cleaning-&-Preparation)

# Setup & Import¶

In [290]:
#import basic system functions
import os
import datetime
from collections import Counter
import itertools

import random

#import libraries for data wrangling
import pandas as pd
import numpy as np

#import libraries for plotting data
import matplotlib.pyplot as plt
import matplotlib.pyplot as mplt

%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pandas.plotting import autocorrelation_plot, lag_plot
from scipy import signal
# If you want a style choose one
#plt.style.use('Solarize_Light2')
#plt.style.use('tableau-colorblind10')
NF_ORANGE = '#ff5a36'
NF_BLUE = '#163251'



#import libraries for time series analysis
from fbprophet import Prophet
from fbprophet.plot import add_changepoints_to_plot
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from pylab import rcParams
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from pmdarima.arima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.api import SimpleExpSmoothing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

#import libraries for statistics
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from fbprophet.diagnostics import performance_metrics
from fbprophet.diagnostics import cross_validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


from IPython.display import Image
from datetime import datetime
import holidays

In [292]:
df_2= pd.read_csv("df_new_2.csv")
df_2_clean= pd.read_csv("data_2_clean.csv")

In [294]:
df_2.drop(columns=['Unnamed: 0'],inplace=True)
df_2_clean.drop(columns=['Unnamed: 0'],inplace=True)

In [296]:
#df= pd.read_csv("/Users/brittarover/Capstone/Meteolytics/meteolytix_Artikelgruppen_Umsatz_verschiedeneStandorte.csv")

In [298]:
df_2_clean.head(2)

,Datum,Weizenbrot,Mischbrot,Vollkornbrot,Stangenbrote,Spezialbrot,Brötchen,Süsse_Brötchen,Herzhafte_Brötchen,KonditoreiBlech,...,Closed,Outlier,New_Years_Eve,Feiertag_DE,Ostern,Holidays_SH,SUMMER_SH,Niederschlag,Sonne_h,Temperatur_max
0,2015-01-01,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1,0,0,1,0.0,1.0,0.0,8.0,0.0,6.2
1,2015-01-02,88.02375,84.63319,115.157621,58.999279,66.687652,505.565569,235.597443,87.759013,66.333083,...,0,0,0,0,0.0,1.0,0.0,2.2,0.7,9.5


In [300]:
df_2_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1979 entries, 0 to 1978
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Datum               1979 non-null   object 
 1   Weizenbrot          1979 non-null   float64
 2   Mischbrot           1979 non-null   float64
 3   Vollkornbrot        1979 non-null   float64
 4   Stangenbrote        1979 non-null   float64
 5   Spezialbrot         1979 non-null   float64
 6   Brötchen            1979 non-null   float64
 7   Süsse_Brötchen      1979 non-null   float64
 8   Herzhafte_Brötchen  1979 non-null   float64
 9   KonditoreiBlech     1979 non-null   float64
 10  Stückgebäck         1979 non-null   float64
 11  Blechkuchen         1979 non-null   float64
 12  Weihnachtsartikel   1979 non-null   float64
 13  Wochentag           1979 non-null   int64  
 14  Tag                 1979 non-null   int64  
 15  Jahr                1979 non-null   int64  
 16  Monat 

In [302]:
df_2.shape

(1979, 33)

## Convert to datetime

In [ ]:
df_2['Datum'] = pd.to_datetime(df_2['Datum'])
df_2_clean['Datum'] = pd.to_datetime(df_2_clean['Datum'])


In [ ]:
df_basis_2=df_2.iloc[:,1:13]
df_basis_2_clean=df_2_clean.iloc[:,1:13]
df_basis_2_dum=df_2_dum.iloc[:,1:13]

## Modify categorical features

In [ ]:
month_dummy= pd.get_dummies(df_2_clean['Monat'], prefix='M', drop_first=True)
weekday_dummy= pd.get_dummies(df_2_clean['Wochentag'], prefix='W', drop_first=True)
season_dummy=pd.get_dummies(df_2_clean['Season'], prefix='S', drop_first=True)

In [ ]:
df_2_dum = df_2_clean.drop(['Tag','Monat','Wochentag','Jahr','Season'], axis=1) 

In [ ]:
df_2_dum = pd.concat([df_2_dum,month_dummy,weekday_dummy,season_dummy], axis=1) 

In [ ]:
df_2_dum.columns

## New Features

### Brückentage

In [ ]:
Brückentage= pd.DataFrame({'date':pd.date_range('2019-05-02', '2019-05-05')})
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2019-05-31', '2019-06-02')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2019-10-04', '2019-10-06')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2019-12-27', '2019-12-30')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2018-04-28', '2018-05-01')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2018-05-10', '2018-05-13')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2018-10-03', '2018-10-08')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2018-12-27', '2018-12-30')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2017-04-30', '2017-05-01')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2017-05-25', '2017-05-28')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2017-09-29', '2017-10-03')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2017-10-28', '2017-10-31')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2017-12-27', '2017-12-30')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2016-12-27', '2016-12-30')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2016-05-05', '2017-05-08')}))
Brückentage = Brückentage.append(pd.DataFrame({'date':pd.date_range('2016-09-30', '2016-10-03')}))


Brückentage['Datum']=Brückentage['date'].apply(lambda x: x.strftime(format = "%Y-%m-%d"))
liste_Brückentage=Brückentage['Datum'].tolist()

In [ ]:
#Definition of new column Brückentage
df_2_dum['Brückentage']=df_2_dum['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in 
                                                liste_Brückentage else 0)
   

### Kieler Woche

In [ ]:
kieler_woche= pd.DataFrame({'date':pd.date_range('2019-06-22', '2019-06-30')})
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2018-06-16', '2018-06-24')}))
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2017-06-17', '2017-06-25')}))
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2016-06-18', '2016-06-26')}))
kieler_woche = kieler_woche.append(pd.DataFrame({'date':pd.date_range('2015-06-20', '2015-06-28')}))

In [ ]:
kieler_woche['Datum']=kieler_woche['date'].apply(lambda x: x.strftime(format = "%Y-%m-%d"))
liste_kw=kieler_woche['Datum'].tolist()

In [ ]:
#Definition of new column Kieler Woche
df_2_dum['Kieler_Woche']=df_2_dum['Datum'].apply(lambda x: 1 if x.strftime(format = "%Y-%m-%d") in liste_kw else 0)

## Timerelated Features for Salesvolumne in the past

#### Values of the previous week and year

In [189]:
articlelist=df_basis_2_dum.columns.tolist()

In [ ]:
#For multiple Columns and also Dataframes
var=['Gesamt']
numbers=[7,364]

for i in var:
    for j in numbers:
       df_2_dum["%s_%d" % (i,j)]=df_2_dum[i].shift(j)
       df_2_dum["%s_%d" % (i,j)].fillna(0)

In [201]:
#For multiple Columns and also Dataframes
#var=['Brötchen','Süsse_Brötchen','Herzhafte_Brötchen']
numbers=[7,364]

for i in articlelist:
    for j in numbers:
       df_2_dum["%s_%d" % (i,j)]=df_2_clean[i].shift(j)
       df_2_dum["%s_%d" % (i,j)].fillna(0)

#### mean and max-Values for Weekday and Monthly Values

**Conclusions**
- `Mean and Maxvalues have no significant impact to the correlation. I need to do it for Months and Weekdays.`


In [ ]:
# Definition on weekldaybase
def Brötchen_mean(x):
    return df_2.groupby('Wochentag')['Brötchen'].mean()[x]
def Brötchen_max(x):
    return df_2.groupby('Wochentag')['Brötchen'].max()[x]


In [ ]:
#For multiple Columns and also Dataframes
var=['Süsse_Brötchen','Herzhafte_Brötchen']


for i in var:
    name=i+ "_mean"
    
    def name(x):
        return df_2.groupby('Wochentag')['%s' % (i)].mean()[x]

    df_2['%s_mean_WD' % (i)]=df_2['Wochentag'].apply(lambda x:  name(x))
                                   

In [ ]:
#For multiple Columns and also Dataframes
var=['Süsse_Brötchen','Herzhafte_Brötchen']


for i in var:
    name=i+ "_max"
    
    def name(x):
        return df_2.groupby('Wochentag')['%s' % (i)].mean()[x]

    df_2['%s_max_WD' % (i)]=df_2['Wochentag'].apply(lambda x:  name(x))

In [ ]:
def Brötchen_mean_M(x):
    return df_2.groupby('Monat')['Brötchen'].mean()[x]
def Brötchen_max_M(x):
    return df_2.groupby('Monat')['Brötchen'].max()[x]


In [ ]:
df_2['Brötchen_mean_M']=df_2['Monat'].apply(lambda x:  Brötchen_mean_M(x))
df_2['Brötchen_max_M']=df_2['Monat'].apply(lambda x:  Brötchen_max_M(x))


In [ ]:
#For multiple Columns and also Dataframes
var=['Süsse_Brötchen','Herzhafte_Brötchen']

for i in var:
    name=i+ "_mean"
    
    def name(x):
        return df_2.groupby('Monat')['%s' % (i)].mean()[x]

    df_2['%s_mean_M' % (i)]=df_2['Monat'].apply(lambda x:  name(x))

In [ ]:
#For multiple Columns and also Dataframes
var=['Süsse_Brötchen','Herzhafte_Brötchen']


for i in var:
    name=i+ "_max"
    
    def name(x):
        return df_2.groupby('Monat')['%s' % (i)].mean()[x]

    df_2['%s_max_M' % (i)]=df_2['Monat'].apply(lambda x:  name(x))
                                   

## Distribution

In [ ]:
# Valuation of agg.Values
df_2.describe().T

## Correlations

In [ ]:
# correlation für Brötchen
basis=num_df.corr(method='pearson')['Brötchen'].sort_values(axis=0, ascending=False)
#basis.plot(kind='bar',figsize=(10,8),title='Korrelation nach Pearson für Brötchen')

In [ ]:
#First we create a new DataFrame with only numerical features
numerics = ['int64', 'float64']

num_df = df_2.select_dtypes(include=numerics)

#Korrelationsanalyse
mask = np.triu( num_df.corr())
plt.figure(figsize=(30,18))
plt.title('Correlation of variables')
ax=sns.heatmap(num_df.corr(),vmax=1.0,annot=True,mask=mask,cmap='coolwarm')
plt.savefig('correlogram_simple.png')
ax.set_ylim((0,30))
plt.show

In [ ]:
df_2.to_csv("df_new_2.csv")

# Checks Timeseries & Setup Modelling

In [235]:
df_newmodel_2 = df_2.set_index('Datum')
df_newmodel_2_dum=df_2_dum.set_index('Datum')

In [ ]:
#df_new_1_index=df_newmodel_1.set_index('Datum')
#df_new_2_index=df_newmodel_2.set_index('Datum')
#df_new_3_index=df_newmodel_1.set_index('Datum')

## Metrics

**Conclusions**
- `I choose MAE and MAPE as Performance Measures for my project.`

I would analyse the results from the different models via the two different metrics mean absolute error **MAE** which is available via Sklearn.metrics and via the metrics mean absolute percentage error called **MAPE**.

There is no inbuilt function in sci-kit learn, so i define a custom function to calculate this measure. I adjusted the commom measure in the way that i do not consider all closed days of a Filiale.


In [ ]:
#calculate mean absolute forecast error which only take actual values unequal to zero in consideration
def mean_absolute_percentage_error_WZ(actual, predicted): 
    
    actual, predicted = np.array(actual), np.array(predicted)
    return np.mean(np.abs((actual - predicted) / actual)) * 100


In [ ]:
#calculate mean absolute percentage forecast error
def mean_absolute_percentage_error(actual, predicted): 
    
    nz_1=(actual==0)
    actual_nz=actual[nz_1==0]
    
    difference_nz=actual-predicted
    difference_nz.dropna()
    return np.mean(np.abs((difference_nz) / actual_nz)) * 100

In [ ]:
#calculate mean absolute percentage forecast error
def mean_absolute_percentage_error_modify(actual, predicted): 
    
    nz_1=(actual==0)
    actual_nz=actual[nz_1==0]
    
    difference_nz=actual-predicted
    difference_nz.dropna()
    return np.mean(np.abs((difference_nz))) 

In [ ]:
from sklearn.metrics import make_scorer
def mae(actual, predict):
    predict = np.array(predict)
    actual = np.array(actual)
    distance = predict - actual
    distance_mean = distance.mean()
    score = np.sqrt(distance_mean)
    return score

In [ ]:
mae_score = make_scorer(mae, greater_is_better = False)

In [ ]:
def regression_results(y_true, y_pred):
# Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)
    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MAPE', round(mean_absolute_percentage_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

## Test-/Train-Split

Split into train and test datasets to build the model on the training dataset and forecast using the test dataset. I decide to use the first 4 years for training and the period since 01.01.2019 for testing. 

In [236]:
# use past 4 years data to forecast the next 15 months until corona

start_train='01.01.2016'
end_train = '31.12.2018'
start_test='01.01.2019'
corona='15.03.2020'

In [237]:
# Basismodel
train_basis_2 = df_newmodel_2_dum[:end_train].copy()
test_basis_2 = df_newmodel_2_dum[end_train:corona].copy()

In [238]:
train_2 = df_newmodel_2_dum[start_train:end_train].copy()
test_2 = df_newmodel_2_dum[start_test:corona].copy()

## Baseline Model

The moving average value can also be used directly to make predictions as a naive model and baseline for further 
on analysis. The fit would has been better if trend and seasonality components of the time series have already been removed or adjusted for.

In [ ]:
# moving average smoothing as a forecast model
from math import sqrt
from numpy import mean
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

In [ ]:
df_results_Baseline= pd.read_csv("df_results_Baseline")
df_results_Baseline.drop(columns=['Unnamed: 0'],inplace=True)
df_results_Baseline

# Random Forest

In [239]:
start_train='01.01.2016'
end_train = '31.12.2018'
start_test='01.01.2019'
corona='15.03.2020'

In [240]:
# Basismodel
train_2 = df_2_dum[365:1461]

In [241]:
test_2 = df_2_dum[1462:1901]

In [242]:
train_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 365 to 1460
Data columns (total 74 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Datum                   1096 non-null   datetime64[ns]
 1   Weizenbrot              1096 non-null   float64       
 2   Mischbrot               1096 non-null   float64       
 3   Vollkornbrot            1096 non-null   float64       
 4   Stangenbrote            1096 non-null   float64       
 5   Spezialbrot             1096 non-null   float64       
 6   Brötchen                1096 non-null   float64       
 7   Süsse_Brötchen          1096 non-null   float64       
 8   Herzhafte_Brötchen      1096 non-null   float64       
 9   KonditoreiBlech         1096 non-null   float64       
 10  Stückgebäck             1096 non-null   float64       
 11  Blechkuchen             1096 non-null   float64       
 12  Weihnachtsartikel       1096 non-null   float6

In [243]:
for i in train_2.iloc[:,1:13].columns:
     train_2[i]=train_2[i].astype(int)

In [244]:
for i in test_2.iloc[:,1:13].columns:
     test_2[i]=test_2[i].astype(int)

In [245]:
y_train_2_1=train_2['Weizenbrot']
y_train_2_2=train_2['Mischbrot']
y_train_2_3=train_2['Vollkornbrot']
y_train_2_4=train_2['Spezialbrot']
y_train_2_5=train_2['Stangenbrote']
y_train_2_6=train_2['Brötchen']
y_train_2_7=train_2['Süsse_Brötchen']
y_train_2_8=train_2['Herzhafte_Brötchen']
y_train_2_9=train_2['KonditoreiBlech']
y_train_2_10=train_2['Stückgebäck']
y_train_2_11=train_2['Blechkuchen']
y_train_2_12=train_2['Weihnachtsartikel']

In [246]:
y_test_2_1=test_2['Weizenbrot']
y_test_2_2=test_2['Mischbrot']
y_test_2_3=test_2['Vollkornbrot']
y_test_2_4=test_2['Spezialbrot']
y_test_2_5=test_2['Stangenbrote']
y_test_2_6=test_2['Brötchen']
y_test_2_7=test_2['Süsse_Brötchen']
y_test_2_8=test_2['Herzhafte_Brötchen']
y_test_2_9=test_2['KonditoreiBlech']
y_test_2_10=test_2['Stückgebäck']
y_test_2_11=test_2['Blechkuchen']
y_test_2_12=test_2['Weihnachtsartikel']

In [247]:
print(X_train_2_6.shape, X_test_2_6.shape) 

(1096, 29) (439, 29)


In [248]:
y_train_2_6.head()

365      0
366    513
367    458
368    286
369    308
Name: Brötchen, dtype: int64

## Basismodel

In [249]:
df_results_Random_Forest = pd.DataFrame( 
                  columns=['Model', 'Label','MAE','MAPE'])

In [ ]:
df_results_Random_Forest.to_csv("df_results_Random_Forest_basis.csv")

In [ ]:
df_results_Random_Forest
#.sort_values(['Label','MAPE']) 

### Weizenbrot

In [250]:
X_train_2_1=train_2.iloc[:,np.r_[14,16,17,19,26:45]]
X_test_2_1=test_2.iloc[:,np.r_[14,16,17,19,26:45]]

In [251]:
X_train=X_train_2_1
X_test=X_test_2_1
y_train=y_train_2_1

In [252]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [253]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [254]:
# Step 3: predict the response for test data
y_pred_rf_2_1= forest.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 23
Average number of nodes 231
Average maximum depth 18


In [255]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_1 = mean_absolute_error(y_test_2_1,y_pred_rf_2_1)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_1,y_pred_rf_2_1)
MAPE_1=mean_absolute_percentage_error(y_test_2_1,y_pred_rf_2_1)
print(f'Der MAE für Weizenbrot beträgt:{MAE_1}')
#print(f'Der MAE_mod für Weizenbrot beträgt:{MAE_mod}')
print(f'Der MAPE für Weizenbrot beträgt:{MAPE_1}')
df_results_Random_Forest.loc['1',:] = ['Random_Forest-Basis','Weizenbrot',MAE_1,MAPE_1]

Der MAE für Weizenbrot beträgt:11.261958997722095
Der MAPE für Weizenbrot beträgt:19.946601615214163


### Mischbrot

In [256]:
X_train_2_2=train_2.iloc[:,np.r_[14,16,17,19,26:45]]
X_test_2_2=test_2.iloc[:,np.r_[14,16,17,19,26:45]]

In [257]:
X_train=X_train_2_2
X_test=X_test_2_2
y_train=y_train_2_2

In [258]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [259]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [260]:
# Step 3: predict the response for test data
y_pred_rf_2_2= forest.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 23
Average number of nodes 233
Average maximum depth 17


In [261]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_2 = mean_absolute_error(y_test_2_2,y_pred_rf_2_2)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_2,y_pred_rf_2_2)
MAPE_2=mean_absolute_percentage_error(y_test_2_2,y_pred_rf_2_2)
print(f'Der MAE für Mischbrot beträgt:{MAE_2}')
#print(f'Der MAE_mod für Weizenbrot beträgt:{MAE_mod}')
print(f'Der MAPE für Mischbrot beträgt:{MAPE_2}')
df_results_Random_Forest.loc['2',:] = ['Random_Forest-Basis','Mischbrot',MAE_2,MAPE_2]

Der MAE für Mischbrot beträgt:13.947608200455582
Der MAPE für Mischbrot beträgt:20.771600093615174


### Vollkornbrot

In [262]:
X_train_2_3=train_2.iloc[:,np.r_[14,16,17,19,26:46]]
X_test_2_3=test_2.iloc[:,np.r_[14,16,17,19,26:46]]

In [263]:
X_train=X_train_2_3
X_test=X_test_2_3
y_train=y_train_2_3

In [264]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [265]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [266]:
# Step 3: predict the response for test data
y_pred_rf_2_3 = forest.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')


Average number of features 24
Average number of nodes 249
Average maximum depth 17


In [267]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_3 = mean_absolute_error(y_test_2_3,y_pred_rf_2_3)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_3,y_pred_rf_2_3)
MAPE_3=mean_absolute_percentage_error(y_test_2_3,y_pred_rf_2_3)
print(f'Der MAE für Vollkornbrot beträgt:{MAE_3}')
#print(f'Der MAE_mod für Vollkornbrot beträgt:{MAE_mod}')
print(f'Der MAPE für Vollkornbrot beträgt:{MAPE_3}')
df_results_Random_Forest.loc['3',:] = ['Random_Forest-Basis','Vollkornbrot',MAE_3,MAPE_3]

Der MAE für Vollkornbrot beträgt:24.59908883826879
Der MAPE für Vollkornbrot beträgt:19.608865755704617


### Spezialbrot

In [268]:
X_train_2_4=train_2.iloc[:,np.r_[14,27:43]]
X_test_2_4=test_2.iloc[:,np.r_[14,27:43]]

In [269]:
X_train_2_4=train_2.iloc[:,np.r_[14:43]]
X_test_2_4=test_2.iloc[:,np.r_[14:43]]

In [270]:
X_train=X_train_2_4
X_test=X_test_2_4
y_train=y_train_2_4

In [271]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [272]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [273]:
# Step 3: predict the response for test data
y_pred_rf_2_4 = forest.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 29
Average number of nodes 1288
Average maximum depth 25


In [274]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_4 = mean_absolute_error(y_test_2_4,y_pred_rf_2_4)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_4,y_pred_rf_2_4)
MAPE_4=mean_absolute_percentage_error(y_test_2_4,y_pred_rf_2_4)
print(f'Der MAE für Spezialbrot beträgt:{MAE_4}')
#print(f'Der MAE_mod für Spezialbrot beträgt:{MAE_mod}')
print(f'Der MAPE für Spezialbrot beträgt:{MAPE_4}')
df_results_Random_Forest.loc['4',:] = ['Random_Forest-Basis','Spezialbrot',MAE_4,MAPE_4]

Der MAE für Spezialbrot beträgt:22.838268792710707
Der MAPE für Spezialbrot beträgt:36.39806982145019


### Stangenbrote

In [275]:
X_train_2_5=train_2.iloc[:,np.r_[14:47]]
X_test_2_5=test_2.iloc[:,np.r_[14:47]]

In [276]:
X_train=X_train_2_5
X_test=X_test_2_5
y_train=y_train_2_5

In [277]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [278]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [279]:
# Step 3: predict the response for test data
y_pred_rf_2_5 = forest.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 33
Average number of nodes 1320
Average maximum depth 26


In [280]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_5 = mean_absolute_error(y_test_2_5,y_pred_rf_2_5)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_5,y_pred_rf_2_5)
MAPE_5=mean_absolute_percentage_error(y_test_2_5,y_pred_rf_2_5)
print(f'Der MAE für Stangenbrote beträgt:{MAE_5}')
#print(f'Der MAE_mod für Spezialbrot beträgt:{MAE_mod}')
print(f'Der MAPE für Stangenbrote beträgt:{MAPE_5}')
df_results_Random_Forest.loc['5',:] = ['Random_Forest-Basis','Stangenbrote',MAE_5,MAPE_5]

Der MAE für Stangenbrote beträgt:14.708428246013668
Der MAPE für Stangenbrote beträgt:48.691470626038374


### Brötchen

In [281]:
X_train_2_6=train_2.iloc[:,np.r_[14,16,17,19,21,22,25:48]]
X_test_2_6=test_2.iloc[:,np.r_[14,16,17,19,21,22,25:48]]

In [282]:
X_train=X_train_2_6
X_test=X_test_2_6
y_train=y_train_2_6



In [283]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

# Step 2: Train the model on the data
forest.fit(X_train,y_train)

# Step 3: predict the response for test data
y_pred_rf_2_6 = forest.predict(X_test)

In [284]:
n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 29
Average number of nodes 1345
Average maximum depth 23


In [285]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

In [286]:
MAE_6 = mean_absolute_error(y_test_2_6,y_pred_rf_2_6)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_6,y_pred_rf_2_6)
MAPE_6=mean_absolute_percentage_error(y_test_2_6,y_pred_rf_2_6)
print(f'Der MAE für Brötchen beträgt:{MAE_6}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Brötchen beträgt:{MAPE_6}')
df_results_Random_Forest.loc['6',:] = ['Random_Forest-Basis','Brötchen',MAE_6,MAPE_6]

Der MAE für Brötchen beträgt:59.52847380410023
Der MAPE für Brötchen beträgt:14.329335816436556


In [ ]:
# Import Support 
from sklearn.model_selection import GridSearchCV

param_search = { 
    'n_estimators': [20, 50, 100,200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [i for i in range(5,50)]
}
#'tscv = TimeSeriesSplit(n_splits=10) /  cv=tscv,

#instatiate GridSearchCV fit model and make prediction

gsearch = GridSearchCV(RandomForestClassifier(), param_grid=param_search, scoring = mae_score)
gsearch.fit(X_train, y_train)
y_pred_new = gsearch.predict(X_test)
best_score = gsearch.best_score_
best_model = gsearch.best_estimator_


In [ ]:
MAE = mean_absolute_error(y_test_2_6,y_pred_new)
MAPE=mean_absolute_percentage_error(y_test_2_6,y_pred_new)
print(f'Der MAE für Brötchen beträgt:{MAE}')
print(f'Der MAPE für Brötchen beträgt:{MAPE}')

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

for ind_tree in gsearch.best_estimator_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

### Süsse Brötchen

In [ ]:
X_train_2_7=train_2.iloc[:,np.r_[14,16,17,19,21,22,25:48]]
X_test_2_7=test_2.iloc[:,np.r_[14,16,17,19,21,22,25:48]]

In [ ]:
X_train=X_train_2_7
X_test=X_test_2_7
y_train=y_train_2_7

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_7 = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_7 = mean_absolute_error(y_test_2_7,y_pred_rf_2_7)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_7,y_pred_rf_2_7)
MAPE_7=mean_absolute_percentage_error(y_test_2_7,y_pred_rf_2_7)
print(f'Der MAE für Süsse Brötchen beträgt:{MAE_7}')
#print(f'Der MAE_mod für Süsse Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Süsse Brötchen beträgt:{MAPE_7}')
df_results_Random_Forest.loc['7',:] = ['Random_Forest-Basis','Süsse Brötchen',MAE_7,MAPE_7]

### Herzhafte Brötchen

In [ ]:
X_train_2_8=train_2.iloc[:,np.r_[14,16,17,19,21,22,25:48]]


X_test_2_8=test_2.iloc[:,np.r_[14,16,17,19,21,22,25:48]]

In [ ]:
X_train_2_8=train_2.iloc[:,np.r_[14:47]]
X_test_2_8=test_2.iloc[:,np.r_[14:47]]

In [ ]:
X_train=X_train_2_8
X_test=X_test_2_8
y_train=y_train_2_8

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_8 = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_8 = mean_absolute_error(y_test_2_8,y_pred_rf_2_8)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_8=mean_absolute_percentage_error(y_test_2_8,y_pred_rf_2_8)
print(f'Der MAE für Herzhafte_Brötchen beträgt:{MAE_8}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Herzhafte_Brötchen beträgt:{MAPE_8}')
df_results_Random_Forest.loc['8',:] = ['Random_Forest-Basis','Herzhafte_Brötchen',MAE_8,MAPE_8]

### KonditoreiBlech

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22]]

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[14:47]]
X_test_2_9=test_2.iloc[:,np.r_[14:47]]

In [ ]:
X_train=X_train_2_9
X_test=X_test_2_9
y_train=y_train_2_9

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_9 = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_9 = mean_absolute_error(y_test_2_9,y_pred_rf_2_9)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_9=mean_absolute_percentage_error(y_test_2_9,y_pred_rf_2_9)
print(f'Der MAE für KonditoreiBlech beträgt:{MAE_9}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für KonditoreiBlech beträgt:{MAPE_9}')
df_results_Random_Forest.loc['9',:] = ['Random_Forest-Basis','KonditoreiBlech',MAE_9,MAPE_9]

### Stückgebäck

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22]]

In [ ]:
X_train_2_10=train_2.iloc[:,np.r_[14:47]]
X_test_2_10=test_2.iloc[:,np.r_[14:47]]

In [ ]:
X_train=X_train_2_10
X_test=X_test_2_10
y_train=y_train_2_10

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_10 = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_10 = mean_absolute_error(y_test_2_10,y_pred_rf_2_10)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_10=mean_absolute_percentage_error(y_test_2_10,y_pred_rf_2_10)
print(f'Der MAE für Stückgebäck beträgt:{MAE_10}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Stückgebäck beträgt:{MAPE_10}')
df_results_Random_Forest.loc['10',:] = ['Random_Forest-Basis','Stückgebäck',MAE_10,MAPE_10]

### Blechkuchen

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22]]

In [ ]:
X_train_2_11=train_2.iloc[:,np.r_[14:47]]
X_test_2_11=test_2.iloc[:,np.r_[14:47]]


In [ ]:
X_train=X_train_2_11
X_test=X_test_2_11
y_train=y_train_2_11

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_11 = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_11 = mean_absolute_error(y_test_2_11,y_pred_rf_2_11)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_11=mean_absolute_percentage_error(y_test_2_11,y_pred_rf_2_11)
print(f'Der MAE für Blechkuchen beträgt:{MAE_11}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Blechkuchen beträgt:{MAPE_11}')
df_results_Random_Forest.loc['11',:] = ['Random_Forest-Basis','Blechkuchen',MAE_11,MAPE_11]

### Weihnachtsartikel

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22]]

In [ ]:
X_train_2_12=train_2.iloc[:,np.r_[14:47]]
X_test_2_12=test_2.iloc[:,np.r_[14:47]]

In [ ]:
X_train=X_train_2_12
X_test=X_test_2_12
y_train=y_train_2_12

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_12 = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_12 = mean_absolute_error(y_test_2_12,y_pred_rf_2_12)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_12=mean_absolute_percentage_error(y_test_2_12,y_pred_rf_2_12)
print(f'Der MAE für Weihnachtsartikel beträgt:{MAE_12}')
#print(f'Der MAE_mod für Weihnachtsartikel beträgt:{MAE_mod}')
print(f'Der MAPE für Weihnachtsartikel beträgt:{MAPE_12}')
df_results_Random_Forest.loc['12',:] = ['Random_Forest-Basis','Weihnachtsartikel',MAE_12,MAPE_12]

## Advanced Model

In [ ]:
train_2.info()

### Weizenbrot

In [229]:
X_train_2_1=train_2.iloc[:,np.r_[14,16,17,19,26:45]]
X_test_2_1=test_2.iloc[:,np.r_[14,16,17,19,26:45]]

In [230]:
X_train=X_train_2_1
X_test=X_test_2_1
y_train=y_train_2_1

In [231]:
# Step 1: Make an instance of the Model
forest_1 = RandomForestClassifier(random_state = 0)

In [232]:
# Step 2: Train the model on the data
forest_1.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [233]:
# Step 3: predict the response for test data
y_pred_rf_2_1_adv= forest_1.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest_1.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 23
Average number of nodes 231
Average maximum depth 18


In [234]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_1_adv = mean_absolute_error(y_test_2_1,y_pred_rf_2_1_adv)
MAPE_1_adv=mean_absolute_percentage_error(y_test_2_1,y_pred_rf_2_1_adv)
print(f'Der MAE für Weizenbrot beträgt:{MAE_1_adv}')
print(f'Der MAPE für Weizenbrot beträgt:{MAPE_1_adv}')
df_results_Random_Forest.loc['13',:] = ['Random_Forest-Advanced','Weizenbrot',MAE_1_adv,MAPE_1_adv]

ValueError: Number of features of the model must match the input. Model n_features is 33 and input n_features is 23 

### Mischbrot

In [183]:
X_train_2_2=train_2.iloc[:,np.r_[14,16,17,19,26:46,48,49]]
X_test_2_2=test_2.iloc[:,np.r_[14,16,17,19,26:46,48,49]]

In [184]:
X_train=X_train_2_2
X_test=X_test_2_2
y_train=y_train_2_2

In [185]:
# Step 1: Make an instance of the Model
forest_2 = RandomForestClassifier(random_state = 0)

In [186]:
# Step 2: Train the model on the data
forest_2.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [187]:
# Step 3: predict the response for test data
y_pred_rf_2_2_adv= forest_2.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest_2.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 26
Average number of nodes 1316
Average maximum depth 26


In [188]:
train_rf_predictions = forest_2.predict(X_train)
train_rf_probs = forest_2.predict_proba(X_train)[:, 1]

rf_predictions = forest_2.predict(X_test)
rf_probs = forest_2.predict_proba(X_test)[:, 1]

MAE_2_adv = mean_absolute_error(y_test_2_2,y_pred_rf_2_2_adv)
MAPE_2_adv=mean_absolute_percentage_error(y_test_2_2,y_pred_rf_2_2_adv)
print(f'Der MAE für Mischbrot beträgt:{MAE_2_adv}')
print(f'Der MAPE für Mischbrot beträgt:{MAPE_2_adv}')
df_results_Random_Forest.loc['14',:] = ['Random_Forest-Advanced','Mischbrot',MAE_2_adv,MAPE_2_adv]

Der MAE für Mischbrot beträgt:15.127562642369021
Der MAPE für Mischbrot beträgt:21.79638301540267


### Vollkornbrot

In [170]:
X_train_2_3=train_2.iloc[:,np.r_[14,16,17,19,26:46,48,49]]
X_test_2_3=test_2.iloc[:,np.r_[14,16,17,19,26:46,48,49]]

In [171]:
X_train=X_train_2_3
X_test=X_test_2_3
y_train=y_train_2_3

In [172]:
# Step 1: Make an instance of the Model
forest_3 = RandomForestClassifier(random_state = 0)

In [173]:
# Step 2: Train the model on the data
forest_3.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [174]:
# Step 3: predict the response for test data
y_pred_rf_2_3_adv = forest_3.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')


Average number of features 33
Average number of nodes 248
Average maximum depth 17


In [175]:
train_rf_predictions = forest_3.predict(X_train)
train_rf_probs = forest_3.predict_proba(X_train)[:, 1]

rf_predictions = forest_3.predict(X_test)
rf_probs = forest_3.predict_proba(X_test)[:, 1]

MAE_3_adv = mean_absolute_error(y_test_2_3,y_pred_rf_2_3_adv)
MAPE_3_adv=mean_absolute_percentage_error(y_test_2_3,y_pred_rf_2_3_adv)
print(f'Der MAE für Vollkornbrot beträgt:{MAE_3_adv}')
print(f'Der MAPE für Vollkornbrot beträgt:{MAPE_3_adv}')
df_results_Random_Forest.loc['15',:] = ['Random_Forest-Advanced','Vollkornbrot',MAE_3_adv,MAPE_3_adv]

Der MAE für Vollkornbrot beträgt:24.170842824601365
Der MAPE für Vollkornbrot beträgt:19.486626956545493


### Spezialbrot

In [176]:
X_train_2_4=train_2.iloc[:,np.r_[14,27:43,48,49]]
X_test_2_4=test_2.iloc[:,np.r_[14,27:43,48,49]]

In [177]:
X_train_2_4=train_2.iloc[:,np.r_[14:43,48,49]]
X_test_2_4=test_2.iloc[:,np.r_[14:43,48,49]]

In [178]:
X_train=X_train_2_4
X_test=X_test_2_4
y_train=y_train_2_4

In [179]:
# Step 1: Make an instance of the Model
forest_4 = RandomForestClassifier(random_state = 0)

In [180]:
# Step 2: Train the model on the data
forest_4.fit(X_train,y_train)

RandomForestClassifier(random_state=0)

In [181]:
# Step 3: predict the response for test data
y_pred_rf_2_4_adv = forest_4.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest_4.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

Average number of features 31
Average number of nodes 1254
Average maximum depth 26


In [182]:
train_rf_predictions = forest_4.predict(X_train)
train_rf_probs = forest_4.predict_proba(X_train)[:, 1]

rf_predictions = forest_4.predict(X_test)
rf_probs = forest_4.predict_proba(X_test)[:, 1]

MAE_4_adv = mean_absolute_error(y_test_2_4,y_pred_rf_2_4_adv)
MAPE_4_adv=mean_absolute_percentage_error(y_test_2_4,y_pred_rf_2_4_adv)
print(f'Der MAE für Spezialbrot beträgt:{MAE_4_adv}')
print(f'Der MAPE für Spezialbrot beträgt:{MAPE_4_adv}')
df_results_Random_Forest.loc['16',:] = ['Random_Forest-Advanced','Spezialbrot',MAE_4_adv,MAPE_4_adv]

Der MAE für Spezialbrot beträgt:23.88382687927107
Der MAPE für Spezialbrot beträgt:38.48349483159684


### Stangenbrote

In [ ]:
X_train_2_5=train_2.iloc[:,np.r_[14:47,53,54]]
X_test_2_5=test_2.iloc[:,np.r_[14:47,53,54]]

In [ ]:
X_train=X_train_2_5
X_test=X_test_2_5
y_train=y_train_2_5

In [ ]:
# Step 1: Make an instance of the Model
forest_5 = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest_5.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_5 = forest_5.predict(X_test)

n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest_5.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_5.predict(X_train)
train_rf_probs = forest_5.predict_proba(X_train)[:, 1]

rf_predictions = forest_5.predict(X_test)
rf_probs = forest_5.predict_proba(X_test)[:, 1]

MAE_5 = mean_absolute_error(y_test_2_5,y_pred_rf_2_5)
MAE_mod=mean_absolute_percentage_error_modify(y_test_2_5,y_pred_rf_2_5)
MAPE_5=mean_absolute_percentage_error(y_test_2_5,y_pred_rf_2_5)
print(f'Der MAE für Stangenbrote beträgt:{MAE_5}')
#print(f'Der MAE_mod für Spezialbrot beträgt:{MAE_mod}')
print(f'Der MAPE für Stangenbrote beträgt:{MAPE_5}')
df_results_Random_Forest.loc['17',:] = ['Random_Forest-Advanced','Stangenbrote',MAE_5,MAPE_5]

### Brötchen

In [ ]:
X_train_2_6=train_2.iloc[:,np.r_[14,16,17,18,19,20,21,22,25:47,57,58,71,72]]
X_test_2_6=test_2.iloc[:,np.r_[14,16,17,18,19,20,21,22,25:47,57,58,71,72]]

In [ ]:
X_train=X_train_2_6
X_test=X_test_2_6
y_train=y_train_2_6



In [ ]:
# Step 1: Make an instance of the Model
forest_6 = RandomForestClassifier(random_state = 0)

# Step 2: Train the model on the data
forest_6.fit(X_train,y_train)

# Step 3: predict the response for test data
y_pred_rf_2_6_adv = forest_6.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest_6.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_6.predict(X_train)
train_rf_probs = forest_6.predict_proba(X_train)[:, 1]

rf_predictions = forest_6.predict(X_test)
rf_probs = forest_6.predict_proba(X_test)[:, 1]

In [ ]:
MAE_6_adv = mean_absolute_error(y_test_2_6,y_pred_rf_2_6_adv)
MAPE_6_adv=mean_absolute_percentage_error(y_test_2_6,y_pred_rf_2_6_adv)
print(f'Der MAE für Brötchen beträgt:{MAE_6_adv}')
print(f'Der MAPE für Brötchen beträgt:{MAPE_6_adv}')
df_results_Random_Forest.loc['42',:] = ['Random_Forest-Advanced_inclGesamt','Brötchen',MAE_6_adv,MAPE_6_adv]

### Süsse Brötchen

In [ ]:
X_train_2_7=train_2.iloc[:,np.r_[14,16,17,18,19,20,21,22,25:47,60,71,72,73]]
X_test_2_7=test_2.iloc[:,np.r_[14,16,17,18,19,20,21,22,25:47,60,71,72,73]]

In [ ]:
X_train=X_train_2_7
X_test=X_test_2_7
y_train=y_train_2_7

In [ ]:
# Step 1: Make an instance of the Model
forest_7 = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest_7.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_7_adv = forest_7.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest_7.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_7.predict(X_train)
train_rf_probs = forest_7.predict_proba(X_train)[:, 1]

rf_predictions = forest_7.predict(X_test)
rf_probs = forest_7.predict_proba(X_test)[:, 1]

MAE_7_adv = mean_absolute_error(y_test_2_7,y_pred_rf_2_7_adv)
MAPE_7_adv=mean_absolute_percentage_error(y_test_2_7,y_pred_rf_2_7_adv)
print(f'Der MAE für Süsse Brötchen beträgt:{MAE_7_adv}')
print(f'Der MAPE für Süsse Brötchen beträgt:{MAPE_7_adv}')
df_results_Random_Forest.loc['43',:] = ['Random_Forest-Advanced_Gesamt','Süsse Brötchen',MAE_7_adv,MAPE_7_adv]

### Herzhafte Brötchen

In [ ]:
X_train_2_8=train_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,61,61,71,72,73]]
X_test_2_8=test_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,61,62,71,72,73]]

In [ ]:
X_train_2_8=train_2.iloc[:,np.r_[14:47]]
X_test_2_8=test_2.iloc[:,np.r_[14:47]]

In [ ]:
X_train=X_train_2_8
X_test=X_test_2_8
y_train=y_train_2_8

In [ ]:
# Step 1: Make an instance of the Model
forest_8 = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest_8.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_8_adv = forest_8.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest_8.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_8.predict(X_train)
train_rf_probs = forest_8.predict_proba(X_train)[:, 1]

rf_predictions = forest_8.predict(X_test)
rf_probs = forest_8.predict_proba(X_test)[:, 1]

MAE_8_adv = mean_absolute_error(y_test_2_8,y_pred_rf_2_8_adv)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_8_adv=mean_absolute_percentage_error(y_test_2_8,y_pred_rf_2_8_adv)
print(f'Der MAE für Herzhafte_Brötchen beträgt:{MAE_8_adv}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Herzhafte_Brötchen beträgt:{MAPE_8_adv}')
df_results_Random_Forest.loc['56',:] = ['Random_Forest-Advanced','Herzhafte_Brötchen',MAE_8,MAPE_8_adv]

### KonditoreiBlech

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22,63,64]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22,63,64]]

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[14:47,63,64]]
X_test_2_9=test_2.iloc[:,np.r_[14:47,63,64]]

In [ ]:
X_train=X_train_2_9
X_test=X_test_2_9
y_train=y_train_2_9

In [ ]:
# Step 1: Make an instance of the Model
forest_9 = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest_9.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_9_adv = forest_9.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest_9.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_9.predict(X_train)
train_rf_probs = forest_9.predict_proba(X_train)[:, 1]

rf_predictions = forest_9.predict(X_test)
rf_probs = forest_9.predict_proba(X_test)[:, 1]

MAE_9_adv = mean_absolute_error(y_test_2_9,y_pred_rf_2_9)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_9_adv=mean_absolute_percentage_error(y_test_2_9,y_pred_rf_2_9)
print(f'Der MAE für KonditoreiBlech beträgt:{MAE_9_adv}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für KonditoreiBlech beträgt:{MAPE_9_adv}')
df_results_Random_Forest.loc['21',:] = ['Random_Forest-Advanced','KonditoreiBlech',MAE_9_adv,MAPE_9_adv]

### Stückgebäck

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22]]

In [ ]:
X_train_2_10=train_2.iloc[:,np.r_[14:47,65,66]]
X_test_2_10=test_2.iloc[:,np.r_[14:47,65,66]]

In [ ]:
X_train=X_train_2_10
X_test=X_test_2_10
y_train=y_train_2_10

In [ ]:
# Step 1: Make an instance of the Model
forest_10 = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest_10.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_10_adv = forest_10.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest_10.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_10.predict(X_train)
train_rf_probs = forest_10.predict_proba(X_train)[:, 1]

rf_predictions = forest_10.predict(X_test)
rf_probs = forest_10.predict_proba(X_test)[:, 1]

MAE_10_adv = mean_absolute_error(y_test_2_10,y_pred_rf_2_10_adv)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_10_adv=mean_absolute_percentage_error(y_test_2_10,y_pred_rf_2_10_adv)
print(f'Der MAE für Stückgebäck beträgt:{MAE_10_adv}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Stückgebäck beträgt:{MAPE_10_adv}')
df_results_Random_Forest.loc['22',:] = ['Random_Forest-Advanced','Stückgebäck',MAE_10_adv,MAPE_10_adv]

### Blechkuchen

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22]]

In [ ]:
X_train_2_11=train_2.iloc[:,np.r_[14:47,67,68]]
X_test_2_11=test_2.iloc[:,np.r_[14:47,67,68]]

In [ ]:
X_train=X_train_2_11
X_test=X_test_2_11
y_train=y_train_2_11

In [ ]:
# Step 1: Make an instance of the Model
forest_11 = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest_11.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_11_adv = forest_11.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest_11.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_11.predict(X_train)
train_rf_probs = forest_11.predict_proba(X_train)[:, 1]

rf_predictions = forest_11.predict(X_test)
rf_probs = forest_11.predict_proba(X_test)[:, 1]

MAE_11_adv = mean_absolute_error(y_test_2_11,y_pred_rf_2_11_adv)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_11_adv=mean_absolute_percentage_error(y_test_2_11,y_pred_rf_2_11_adv)
print(f'Der MAE für Blechkuchen beträgt:{MAE_11_adv}')
print(f'Der MAPE für Blechkuchen beträgt:{MAPE_11_adv}')
df_results_Random_Forest.loc['23',:] = ['Random_Forest-Basis','Blechkuchen',MAE_11_adv,MAPE_11_adv]

### Weihnachtsartikel

In [ ]:
X_train_2_9=train_2.iloc[:,np.r_[26,42,22,69,70]]
X_test_2_9=test_2.iloc[:,np.r_[26,42,22,69,70]]

In [ ]:
X_train_2_12=train_2.iloc[:,np.r_[14:47,69,70]]
X_test_2_12=test_2.iloc[:,np.r_[14:47,60,70]]

In [ ]:
X_train=X_train_2_12
X_test=X_test_2_12
y_train=y_train_2_12

In [ ]:
# Step 1: Make an instance of the Model
forest_12 = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest_12.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_12_adv = forest_12.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest_12.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest_12.predict(X_train)
train_rf_probs = forest_12.predict_proba(X_train)[:, 1]

rf_predictions = forest_12.predict(X_test)
rf_probs = forest_12.predict_proba(X_test)[:, 1]

MAE_12_adv = mean_absolute_error(y_test_2_12,y_pred_rf_2_12_adv)
MAPE_12_adv=mean_absolute_percentage_error(y_test_2_12,y_pred_rf_2_12_adv)
print(f'Der MAE für Weihnachtsartikel beträgt:{MAE_12_adv}')
print(f'Der MAPE für Weihnachtsartikel beträgt:{MAPE_12_adv}')
df_results_Random_Forest.loc['24',:] = ['Random_Forest-Basis','Weihnachtsartikel',MAE_12,MAPE_12_adv]

## Advanced Model with adjusted Trainset

**Conclusions**
- `I check if the replacing of the zero-Sales has an impact to the predictions. I do it first for all articlegroups which belongs to Brötchen. `
- `There is no significant impact so I skip to this and have a lot to feature importance of the different labels. `


### Brötchen

In [ ]:
train_2['Brötchen']=train_2['Brötchen'].apply(lambda x: x if x> 0 else None)

In [ ]:
train_2['Brötchen'].fillna(method='ffill',inplace=True)
train_2['Brötchen'].fillna(method='bfill',inplace=True)
train_2['Brötchen']=train_2['Brötchen'].astype(int)

In [ ]:
X_train_2_6=train_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,58,70,71]]
X_test_2_6=test_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,58,70,71]]

In [ ]:
X_train=X_train_2_6
X_test=X_test_2_6
y_train=train_2['Brötchen']



In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_6_adv = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

In [ ]:
MAE_6_adv = mean_absolute_error(y_test_2_6,y_pred_rf_2_6_adv)
MAPE_6_adv=mean_absolute_percentage_error(y_test_2_6,y_pred_rf_2_6_adv)
print(f'Der MAE für Brötchen beträgt:{MAE_6_adv}')
print(f'Der MAPE für Brötchen beträgt:{MAPE_6_adv}')
df_results_Random_Forest.loc['54',:] = ['Random_Forest-Advanced_Closedrepl','Brötchen',MAE_6_adv,MAPE_6_adv]

### Süsse Brötchen

In [ ]:
train_2['Süsse_Brötchen']=train_2['Süsse_Brötchen'].apply(lambda x: x if x> 0 else None)

In [ ]:
train_2['Süsse_Brötchen'].fillna(method='ffill',inplace=True)
train_2['Süsse_Brötchen'].fillna(method='bfill',inplace=True)

In [ ]:
train_2['Süsse_Brötchen']=train_2['Süsse_Brötchen'].astype(int)

In [ ]:
X_train_2_7=train_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,60,70,71]]
X_test_2_7=test_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,60,70,71]]

In [ ]:
X_train=X_train_2_7
X_test=X_test_2_7
y_train=train_2['Süsse_Brötchen']

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_7_adv = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_7_adv = mean_absolute_error(y_test_2_7,y_pred_rf_2_7_adv)
MAPE_7_adv=mean_absolute_percentage_error(y_test_2_7,y_pred_rf_2_7_adv)
print(f'Der MAE für Süsse Brötchen beträgt:{MAE_7_adv}')
print(f'Der MAPE für Süsse Brötchen beträgt:{MAPE_7_adv}')
df_results_Random_Forest.loc['55',:] = ['Random_Forest-Advanced_Closedrepl','Süsse Brötchen',MAE_7_adv,MAPE_7_adv]

### Herzhafte Brötchen

In [ ]:
train_2['Herzhafte_Brötchen']=train_2['Herzhafte_Brötchen'].apply(lambda x: x if x> 0 else None)

In [ ]:
train_2['Herzhafte_Brötchen'].fillna(method='ffill',inplace=True)
train_2['Herzhafte_Brötchen'].fillna(method='bfill',inplace=True)

In [ ]:
train_2['Herzhafte_Brötchen']=train_2['Herzhafte_Brötchen'].astype(int)

In [ ]:
X_train_2_8=train_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,61,61]]
X_test_2_8=test_2.iloc[:,np.r_[14,16,17,19,21,22,25:47,61,62]]

In [ ]:
X_train_2_8=train_2.iloc[:,np.r_[14:47]]
X_test_2_8=test_2.iloc[:,np.r_[14:47]]

In [ ]:
X_train=X_train_2_8
X_test=X_test_2_8
y_train=train_2['Herzhafte_Brötchen']

In [ ]:
# Step 1: Make an instance of the Model
forest = RandomForestClassifier(random_state = 0)

In [ ]:
# Step 2: Train the model on the data
forest.fit(X_train,y_train)

In [ ]:
# Step 3: predict the response for test data
y_pred_rf_2_8_adv = forest.predict(X_test)

In [ ]:
n_nodes = []
max_depths = []
n_features=[]

In [ ]:
for ind_tree in forest.estimators_:
    n_features.append(ind_tree.tree_.n_features)
    n_nodes.append(ind_tree.tree_.node_count)
    max_depths.append(ind_tree.tree_.max_depth)
    
print(f'Average number of features {int(np.mean(n_features))}')
print(f'Average number of nodes {int(np.mean(n_nodes))}')
print(f'Average maximum depth {int(np.mean(max_depths))}')

In [ ]:
train_rf_predictions = forest.predict(X_train)
train_rf_probs = forest.predict_proba(X_train)[:, 1]

rf_predictions = forest.predict(X_test)
rf_probs = forest.predict_proba(X_test)[:, 1]

MAE_8_adv = mean_absolute_error(y_test_2_8,y_pred_rf_2_8_adv)
#MAE_mod=mean_absolute_percentage_error_modify(y_test_2_8,y_pred_rf_2_8)
MAPE_8_adv=mean_absolute_percentage_error(y_test_2_8,y_pred_rf_2_8_adv)
print(f'Der MAE für Herzhafte_Brötchen beträgt:{MAE_8_adv}')
#print(f'Der MAE_mod für Brötchen beträgt:{MAE_mod}')
print(f'Der MAPE für Herzhafte_Brötchen beträgt:{MAPE_8_adv}')
df_results_Random_Forest.loc['56',:] = ['Random_Forest-Closedrepl','Herzhafte_Brötchen',MAE_8,MAPE_8_adv]

## Feature Importance

**Conclusions**
- `So lets have a lot to feature importance. Some of the results are rather surprising. A couple of article groups seems to be dependent of the sales figures of Weizenbrot.`
- `So I decide to fit the belonging models again and consider this features.`


### Weizenbrot

In [ ]:
imp_2_1 = forest_1.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_1)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Weizenbrot')
plt.barh(range(len(indices[17:])), imp_2_1[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[17:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### Mischbrot

In [ ]:
imp_2_2 = forest_2.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_2)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Mischbrot')
plt.barh(range(len(indices[17:])), imp_2_2[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[17:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### Vollkornbrot

In [ ]:
imp_2_3 = forest_3.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_3)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Vollkornbrot')
plt.barh(range(len(indices[17:])), imp_2_3[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[17:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### Spezialbrot

In [ ]:
imp_2_4 = forest_4.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_4)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Spezialbrot')
plt.barh(range(len(indices[17:])), imp_2_4[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[17:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### Stangenbrote

In [ ]:
imp_2_5 = forest_5.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_5)

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Feature Importances for Stangenbrote')
plt.barh(range(len(indices[25:])), imp_2_5[indices[25:]], color='b', align='center')
plt.yticks(range(len(indices[25:])), [features[i] for i in indices[25:]])
plt.xlabel('Relative Importance')
plt.show()

### Brötchen

In [ ]:
imp_2_6 = forest_6.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_6)

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Feature Importances for Brötchen')
plt.barh(range(len(indices[17:])), imp_2_6[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[17:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### Süsse Brötchen

In [ ]:
imp_2_7 = forest_7.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_7)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Süsse Brötchen')
plt.barh(range(len(indices[17:])), imp_2_7[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[17:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### Herzhafte Brötchen

In [ ]:
imp_2_8 = forest_8.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_8)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Herzhafte Brötchen')
plt.barh(range(len(indices[17:])), imp_2_8[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[17:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### KonditoreiBlech

In [ ]:
imp_2_9 = forest_9.feature_importances_
features = train_2.iloc[:,14:47].columns
indices = np.argsort(imp_2_9)

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Feature Importances for KonditoreiBlech')
plt.barh(range(len(indices[25:])), imp_2_9[indices[17:]], color='b', align='center')
plt.yticks(range(len(indices[25:])), [features[i] for i in indices[17:]])
plt.xlabel('Relative Importance')
plt.show()

### Stückgebäck

In [ ]:
imp_2_10 = forest_10.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_10)

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Feature Importances for Stückgebäck')
plt.barh(range(len(indices[25:])), imp_2_10[indices[25:]], color='b', align='center')
plt.yticks(range(len(indices[25:])), [features[i] for i in indices[25:]])
plt.xlabel('Relative Importance')
plt.show()

### Blechkuchen

In [ ]:
imp_2_11 = forest_11.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_11)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Blechkuchen')
plt.barh(range(len(indices[25:])), imp_2_11[indices[25:]], color='b', align='center')
plt.yticks(range(len(indices[25:])), [features[i] for i in indices[25:]])
plt.xlabel('Relative Importance')
plt.show()

### Weihnachtsartikel

In [ ]:
imp_2_12 = forest_12.feature_importances_
features = train_2.iloc[:,14:].columns
indices = np.argsort(imp_2_12)

plt.figure(figsize=(15,5))
plt.title('Feature Importances for Weihnachtsartikel')
plt.barh(range(len(indices[25:])), imp_2_12[indices[25:]], color='b', align='center')
plt.yticks(range(len(indices[25:])), [features[i] for i in indices[25:]])
plt.xlabel('Relative Importance')
plt.show()

## Further optimazation due to performance plots

## More Models to consider?

### Time-Series Split

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor 
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

### Model Comparison for the total sales of the store

In [ ]:
X_train=train_2.iloc[:,14:]
y_train=train_2['Gesamt']

In [ ]:
models = []
#models.append(('LR', LinearRegression()))
models.append(('NN', MLPRegressor(solver = 'lbfgs')))  #neural network
models.append(('KNN', KNeighborsRegressor())) 
models.append(('RF', RandomForestRegressor(n_estimators = 10))) # Ensemble method - collection of many decision trees
models.append(('SVR', SVR(gamma='auto'))) # kernel = linear
# Evaluate each model in turn
results = []
names = []
for name, model in models:
    # TimeSeries Cross validation
 tscv = TimeSeriesSplit(n_splits=5)
    
 cv_results = cross_val_score(model, X_train, y_train, cv=tscv, scoring='r2')
 results.append(cv_results)
 names.append(name)
 print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
# Compare Algorithms
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

In [ ]:
models = []
#models.append(('LR', LinearRegression()))
models.append(('NN', MLPRegressor(solver = 'lbfgs')))  #neural network
models.append(('KNN', KNeighborsRegressor())) 
models.append(('RF', RandomForestRegressor(n_estimators = 10))) # Ensemble method - collection of many decision trees
models.append(('SVR', SVR(gamma='auto'))) # kernel = linear
# Evaluate each model in turn
results = []
names = []
for name, model in models:
    # TimeSeries Cross validation
 tscv = TimeSeriesSplit(n_splits=5)
    
 cv_results = cross_val_score(model, X_train, train_2['Brötchen'], cv=tscv, scoring='r2')
 results.append(cv_results)
 names.append(name)
 print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))
    
# Compare Algorithms
plt.boxplot(results, labels=names)
plt.title('Algorithm Comparison')
plt.show()

# Rocket

In [ ]:
X_train_R=train_2.iloc[:,14:].copy()

In [ ]:
X_train_R

In [ ]:
y_train=train_2['Brötchen']

In [ ]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV

In [ ]:
#from sktime.datasets import load_arrow_head  # univariate dataset
from sktime.transformers.series_as_features.rocket import Rocket

In [ ]:
rocket = Rocket(num_kernels=10000, random_state=111) 
rocket.fit(X_train_R)
X_train_transform = rocket.transform(X_train_R)
X_train_transform.shape

In [ ]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

In [ ]:
X_test_transform = rocket.transform(X_test)
classifier.score(X_test_transform, y_test)